In [4]:
#did not work
#!pip install -q git+https://github.com/tensorflow/examples.git
pip install tensorflow_datasets

SyntaxError: invalid syntax (<ipython-input-4-be366fc91385>, line 3)

In [9]:
#coding in tensor flow 1.0
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset


In [10]:
# mnist = tfds.load('mnist')
#data set in C:\Users\Acer\tensorflow_datasets\mnist\3.0.1

<h2>steps</h2>


input > weight 2 > hidden layer 1 (activation function) > weights > hidden layer 2 (activation function) > weights > output layer

compare output with intended output > cost function (cross entropy)
optimization function (optimizer) > minimize cost (adamOptimizer ... SGD, AdaGrad

back propagation

feed forward + backprop = epoch

code from https://pythonprogramming.net/train-test-tensorflow-deep-learning-tutorial/
(old version, does not properly work)

In [44]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 5s 0us/step


In [36]:
#dataset = tfds.load(name="mnist", split=tfds.Split.TRAIN)

#dataset, info = tfds.load("mnist", with_info=True, as_supervised=True)

#print(dataset)
#mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

# one hot encode for 10 MNIST classes
#def my_one_hot(feature, label):
#    return feature, tf.one_hot(label, depth=10)

# load your data from tfds
#mnist_train, train_info = tfds.load(name="mnist", with_info=True, as_supervised=True, split=tfds.Split.TRAIN)

# convert your labels in one-hot
#mnist_train = mnist_train.map(my_one_hot)

# you can batch your data here
#mnist_train = mnist_train.batch(8)

In [45]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [46]:
#tree hidden layers
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

#number of classes
n_classes = 10

#not enougth of ram to load all at same time
batch_size = 100

#transform the matrix to a vector
# for tf 1.0
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

#in 2.0
#https://gimoonnam.github.io/machinelearning/tensorFlow_1/
#https://www.kaggle.com/general/201962

In [47]:
def neural_network_model(data):
    
    initializer = tf.random_normal_initializer()
    
    # input_data * weight + biases
    #first hidden layer
    hidden_1_layer = {'weights':  tf.Variable(tf.random_normal(shape=[784, n_nodes_hl1], dtype=tf.float32)),
                         'biases': tf.Variable(tf.random_normal(shape=[n_nodes_hl1], dtype=tf.float32))}
    
    #second hidden layer
    hidden_2_layer = {'weights':  tf.Variable(tf.random_normal(shape=[n_nodes_hl1, n_nodes_hl2], dtype=tf.float32)),
                         'biases': tf.Variable(tf.random_normal(shape=[n_nodes_hl2], dtype=tf.float32))}
        
    #third hidden layer
    hidden_3_layer = {'weights':  tf.Variable(tf.random_normal(shape=[n_nodes_hl2, n_nodes_hl3], dtype=tf.float32)),
                         'biases': tf.Variable(tf.random_normal(shape=[n_nodes_hl3], dtype=tf.float32))}
    
    #output hidden layer
    output_layer = {'weights':  tf.Variable(tf.random_normal(shape=[n_nodes_hl3, n_classes])),
                         'biases': tf.Variable(tf.random_normal(shape=[n_classes]))}
    
    
    
    #model input_data * weight + biases
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) +  output_layer['biases']
    
    return output

In [57]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    #tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=Y)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(prediction, y))
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #cycles feed forward + back prop
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        # make an iterator
        train_iterator = tf.data.make_initializable_iterator(mnist_train)
        next_element = train_iterator.get_next()
        
        
        for epoch in range(hm_epochs):
            opoch_loss = 0
            
            sess.run(train_iterator.initializer)
            epoch_x, epoch_y = sess.run(next_element)
            
            for _ in range(int(train_info.splits["train"].num_examples/batch_size)):
                #sess.run(optimizer, feed_dict={x: batch_train_x, y: batch_train_y})
                epoch_x, epoch_y = sess.run(x_train.train.next_batch(batch_size))
                #epoch_x, epoch_y = mnist_train.batch(batch_size)
                #epoch_x, epoch_y = next_batch(batch_size, mnist_train, labels)
                _, c = sess.run([optimizer, cost], feed_dict = {x : epoch_x, y : epoch_y})
                epoch_loss += c 
                
            print("Epoch", epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('accuracy:', accuracy.eval({x:mnist.images, y:mnist.test.labels}))
        

In [58]:
#tf.enable_eager_execution() 
train_neural_network(x)


AttributeError: 'MapDataset' object has no attribute 'train'

In [14]:
initializer = tf.random_normal_initializer()
tf.Variable(initializer(shape=[5], dtype=tf.float32))

<tf.Variable 'Variable_5:0' shape=(5,) dtype=float32_ref>

In [39]:
mnist_train.


<BatchDataset shapes: ((?, 28, 28, 1), (?, 10)), types: (tf.uint8, tf.float32)>

In [8]:
prediction = neural_network_model(x)
#tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=Y)
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(prediction, y))

optimizer = tf.train.AdamOptimizer().minimize(cost)

#cycles feed forward + back prop
hm_epochs = 10

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # make an iterator
    train_iterator = tf.data.make_initializable_iterator(mnist_train)
    next_element = train_iterator.get_next(mnist_train, batch_size = batch_size)

    for epoch in range(hm_epochs):
        opoch_loss = 0

        sess.run(train_iterator.initializer)
        epoch_x, epoch_y = sess.run(next_element)

        for _ in range(int(train_info.splits["train"].num_examples/batch_size)):
            #sess.run(optimizer, feed_dict={x: batch_train_x, y: batch_train_y})
            #epoch_x, epoch_y = dataset["train"].batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {x : epoch_x, y : epoch_y})
            epoch_loss += c 

        print("Epoch", epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)

    correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('accuracy:', accuracy.eval({x:mnist.images, y:mnist.test.labels}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


AttributeError: 'Iterator' object has no attribute 'batch'